## Jupyter Notebook to Aid Development

In [ ]:
import requests

import pandas as pd

url = "https://v1.data.uccf.io/api/christian-unions/expand"
response = requests.get(url)
data = response.json()

In [12]:
print(data)

[{'id': 4, 'name': 'Aberdeen CU', 'full_name': 'Aberdeen University Christian Union', 'campus': None, 'abbreviation': 'AUCU', 'url': 'https://linktr.ee/abdncu', 'website': 'https://linktr.ee/abdncu', 'facebook': 'https://www.facebook.com/abdncu/', 'twitter': None, 'instagram': 'https://www.instagram.com/abdncu', 'institutions': [{'id': 1, 'name': 'Aberdeen, University of', 'postcode': 'AB24 3FX', 'geocode': '57.165019900, -2.099122300', 'region': {'id': 1, 'name': 'Scotland'}}]}, {'id': 5, 'name': 'Abertay CU', 'full_name': 'Abertay University Christian Union', 'campus': None, 'abbreviation': None, 'url': None, 'website': None, 'facebook': 'http://www.facebook.com/AbertayCU', 'twitter': None, 'instagram': 'https://www.instagram.com/abertaycu/', 'institutions': [{'id': 2, 'name': 'Abertay, University of', 'postcode': 'DD1 1HG', 'geocode': '56.46315555, -2.974448773', 'region': {'id': 1, 'name': 'Scotland'}}]}, {'id': 6, 'name': 'Aberystwyth CU', 'full_name': 'Aberystwyth University Chri

In [13]:
# convert data to dataframe
datadf = pd.DataFrame(data)
display(datadf)

,id,name,full_name,campus,abbreviation,url,website,facebook,twitter,instagram,institutions
0,4,Aberdeen CU,Aberdeen University Christian Union,None,AUCU,https://linktr.ee/abdncu,https://linktr.ee/abdncu,https://www.facebook.com/abdncu/,None,https://www.instagram.com/abdncu,"[{'id': 1, 'name': 'Aberdeen, University of', ..."
1,5,Abertay CU,Abertay University Christian Union,None,None,None,None,http://www.facebook.com/AbertayCU,None,https://www.instagram.com/abertaycu/,"[{'id': 2, 'name': 'Abertay, University of', '..."
2,6,Aberystwyth CU,Aberystwyth University Christian Union,None,None,None,None,https://www.facebook.com/aberystwythcu,None,https://www.instagram.com/aberystwythcu/,"[{'id': 3, 'name': 'Aberystwyth University', '..."
3,8,"Anglia Ruskin Uni, Cambridge CU","Anglia Ruskin University, Cambridge Christian ...",Cambridge,None,None,None,None,None,https://www.instagram.com/anglia_ruskin_cu,"[{'id': 707, 'name': 'Anglia Ruskin University..."
4,868,Arden School of Theatre CU,Arden School of Theatre Christian Union,None,None,None,None,None,None,https://www.instagram.com/uom_christianunion/,"[{'id': 830, 'name': 'Arden School of Theatre'..."
...,...,...,...,...,...,...,...,...,...,...,...
184,136,Wolverhampton CU,Wolverhampton University Christian Union,Wolverhampton,None,None,None,None,None,https://www.instagram.com/wlv_cu,"[{'id': 186, 'name': 'Wolverhampton, Universit..."
185,137,Worcester CU,Worcester University Christian Union,None,WORCCU,None,None,https://www.facebook.com/uniworccu/,None,https://www.instagram.com/cuworc_,"[{'id': 187, 'name': 'Worcester, University of..."
186,275,Wrexham CU,Wrexham Christian Union,None,None,None,None,https://www.facebook.com/GlyndwrUniCU/,None,https://www.instagram.com/glyndwr.uni.cu,"[{'id': 776, 'name': 'Wrexham University', 'po..."
187,232,Writtle College CU,Writtle College Christian Union,None,None,None,None,None,None,https://www.instagram.com/wucchristianunion/,"[{'id': 188, 'name': 'Writtle College', 'postc..."


Notes:
- some have abbreviations, some don't
- socials and websites can be lumped together
- host institutions has a whole other dataset

- use Dash as I'm familiar with plotly which is great for handling data
- friendly format (a interactive table - filter based on location)
- search function

prioritise showing name > full name (when hover) > click to see socials/website
institution implementation as future extenstion.

- click to show details
- handle links with clicks?

- idea implement cloropleth map with click functions


In [14]:
from dash import Dash, html

app = Dash()

app.layout = [html.Div(children='Hello World')]

if __name__ == '__main__':
    app.run(debug=True)

In [77]:
institutions = datadf['institutions']


3.14
189


In [ ]:
# first run through to get latitudes and longitudes and dots on the map
import dash
from dash import dcc, html
import plotly.graph_objects as go

# Extract institution data
institutions = datadf['institutions']

# Parse geocodes
lats = []
lons = []
names = []

for i in range(len(institutions)):
    if institutions[i]['geocode'] is not None:
        lat, lon = (institutions[i]['geocode'].split(','))
        lats.append(float(lat))
        lons.append(float(lon))
    else:
        lats.append(0)
        lons.append(0)
    names.append(institutions[i]['name'])

# Create the Dash app
app = dash.Dash(__name__)

# Create the map figure
fig = go.Figure(go.Scattermapbox(
    lat=lats,
    lon=lons,
    mode='markers',
    marker=dict(size=12),
    text=names,
    hoverinfo='text'
))

fig.update_layout(
    mapbox=dict(
        style='open-street-map',  # Free, no token needed
        center=dict(lat=sum(lats)/len(lats), lon=sum(lons)/len(lons)),
        zoom=5
    ),
    margin=dict(l=0, r=0, t=0, b=0),
    height=600
)

app.layout = html.Div([
    html.H1('UK Institutions Map'),
    dcc.Graph(figure=fig)
])

if __name__ == '__main__':
    app.run_server(debug=True)

In [114]:
institutions = datadf['institutions']

print(institutions[0])
print(datadf['full_name'][0])

{'id': 1, 'name': 'Aberdeen, University of', 'postcode': 'AB24 3FX', 'geocode': '57.165019900, -2.099122300', 'region': {'id': 1, 'name': 'Scotland'}, 'website': 'https://linktr.ee/abdncu', 'facebook': 'https://www.facebook.com/abdncu/', 'instagram': 'https://www.instagram.com/abdncu'}
Aberdeen University Christian Union


In [117]:
# institution is compiled here:
institutions = datadf['institutions']

for i in range(len(datadf)):
    institutions[i]["full_name"] = datadf['full_name'][i]
    # check if the social links exist
    for socials in ['website', 'twitter', 'facebook', 'instagram']:
        if datadf[socials][i] != None:
            # if exists, add to the insitutions dict
            institutions[i][socials] = datadf[socials][i]

print(institutions[0])
print(institutions[0]['full_name'])

{'id': 1, 'name': 'Aberdeen, University of', 'postcode': 'AB24 3FX', 'geocode': '57.165019900, -2.099122300', 'region': {'id': 1, 'name': 'Scotland'}, 'website': 'https://linktr.ee/abdncu', 'facebook': 'https://www.facebook.com/abdncu/', 'instagram': 'https://www.instagram.com/abdncu', 'full_name': 'Aberdeen University Christian Union'}
Aberdeen University Christian Union


In [120]:
# second run through to get the click function to output url data
import dash
from dash import dcc, html
import plotly.graph_objects as go

# Parse geocodes
lats = []
lons = []
names = []
full_names = []

for i in range(len(institutions)):
    if institutions[i]['geocode'] is not None:
        lat, lon = (institutions[i]['geocode'].split(','))
        lats.append(float(lat))
        lons.append(float(lon))
    else:
        lats.append(0)
        lons.append(0)
    names.append(institutions[i]['name'])

# Create the Dash app
app = dash.Dash(__name__)

# Create the map figure
fig = go.Figure(go.Scattermapbox(
    lat=lats,
    lon=lons,
    mode='markers',
    marker=dict(size=12),
    text=names,
    hoverinfo='text'
))

fig.update_layout(
    mapbox=dict(
        style='open-street-map',  # Free, no token needed
        center=dict(lat=sum(lats)/len(lats), lon=sum(lons)/len(lons)),
        zoom=5
    ),
    margin=dict(l=0, r=0, t=0, b=0),
    height=600,
    clickmode='event+select'  # Enable click events
)

app.layout = html.Div([
    html.H1('UK Institutions Map'),
    dcc.Graph(id='map', figure=fig),
    html.Div(id='institution-info', style={
        'padding': '20px',
        'marginTop': '20px',
        'border': '1px solid #ddd',
        'borderRadius': '5px'
    })
])

@app.callback(
    Output('institution-info', 'children'),
    Input('map', 'clickData')
)

def display_click_data(clickData):
    if clickData is None:
        return html.P('Click on a marker to see institution details and links')
    
    # Get the clicked institution index position
    point_index = clickData['points'][0]['pointIndex']
    inst = institutions[point_index]
    
    # Create the links display
    links = [
        html.H3(inst['name']),
        html.H3(inst['full_name']),
        html.P(f"Postcode: {inst['postcode']}"),
        html.P(f"Region: {inst['region']['name']}"),
        html.Hr(),
        html.H4('Links:'),

    ]

    # Define all possible social links with their icons and labels
    social_config = {
        'website': {'icon': '🌐', 'label': 'Official Website'},
        'twitter': {'icon': '🐦', 'label': 'Twitter'},
        'instagram': {'icon': '📷', 'label': 'Instagram'},
        'facebook': {'icon': '📘', 'label': 'Facebook'},
    }

    # logic to only show existing links
    found_links = False

    # logic to add only existing links
    for key, config in social_config.items():
        if key in inst:
            links.append(
                html.A(f"{config['icon']} {config['label']}", 
                       href=inst[key], 
                       target='_blank',
                       style={'display': 'block', 'margin': '10px 0', 'fontSize': '16px'})
            )
            found_links = True

    if not found_links:
        links.append(html.P("No links available for this institution."))
            
    
    return html.Div(links)

if __name__ == '__main__':
    app.run_server(debug=True)